Для группированных данных:

In [1]:
import pandas as pd
from  matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


формат загруженного датафрейма: ["cookie_id_num", "vacancy_id_num", "Rating"], агрегация - макс

In [3]:
df_cut = pd.read_csv('/content/drive/MyDrive/Hackaton 04.2023/Aggregated_data.csv',index_col=False)

Для трейн/теста выбираем пользователей у которых максимальная оценка по всем вакансиям > 4, посчитали зрителей и тех, кто в активном поиске


In [4]:
df_temp = df_cut[['cookie_id_num','Rating']].groupby(['cookie_id_num'])['Rating'].max().reset_index()
print(f"Viewers: {len(df_temp[df_temp['Rating']<5])}")
print(f"Searchers: {len(df_temp[df_temp['Rating']>4])}")
ratio = len(df_temp[df_temp['Rating']>4])/len(df_temp)
print(f'searches/all_users ratio = {ratio}')

good_users_set = set(df_temp[df_temp['Rating']>4]['cookie_id_num'])
df_good_users = df_cut[df_cut['cookie_id_num'].isin(good_users_set)]



Viewers: 209802
Searchers: 120378
searches/all_users ratio = 0.36458295475195346


Фильтруем только хорошие оценки для теста(>6), оставляем пользователей, у которых "num_high_events = 9" - более 9 высоких оценок

In [5]:
df_high_rates = df_good_users[df_good_users['Rating']>6]
counts = df_high_rates.groupby(['cookie_id_num']).count().reset_index()
num_high_events = 9
print(f"users with more than {num_high_events} high rates: {len(counts[counts['vacancy_id_num']>num_high_events])}")
users_high_rates = set(counts[counts['vacancy_id_num']>9]['cookie_id_num'])
#38fb5cecd7624620a1030273bb99d730
#users_high_rates

users with more than 9 high rates: 1790


Забрать 5 лучших для теста, остальное останется в трейне

In [6]:
def get_test(df: pd.DataFrame, uhr: set,n_top=5):

    # Filter the input DataFrame to only include high-rating users
    df_uhr = df[df['cookie_id_num'].isin(uhr)]
    df_sorted = df_uhr.sort_values(['cookie_id_num', 'Rating'], ascending=False)
    
    df_top = df_sorted.groupby('cookie_id_num').head(n_top).reset_index(drop=True)
    # Group the filtered DataFrame by 'uid' and aggregate the 'iid' and 'rating' columns
    df_test = df_top.groupby('cookie_id_num').agg({'vacancy_id_num':list,'Rating':list}).reset_index()

    return df_test[['cookie_id_num','vacancy_id_num']], df_top


Считаем тест (выводится 1й вариант по формату и 2й более удобный для нахождения трейна далее)

In [7]:
df_test_list_top,df_test_top = get_test(df_cut,users_high_rates)

Считываем исходный дф, мапим, переименовываем

In [8]:
data = pd.read_parquet('/content/drive/MyDrive/Hackaton 04.2023/train_mfti (1).parquet', engine='pyarrow')



In [9]:
df = data.copy()
df['Rating'] = df['event_type'].map({'show_vacancy': 0,
                                     'preview_click_vacancy': 1,
                                     'click_favorite': 2,
                                     'preview_click_favorite': 3,
                                     'click_response': 4,
                                     'preview_click_response': 5,
                                     'click_contacts': 6,
                                     'preview_click_contacts': 7,
                                     'click_phone': 8,
                                     'preview_click_phone': 9})
df['vacancy_id_num'] = df['vacancy_id_'].astype('int64')
df['cookie_id_num'] = df['cookie_id']
df['user_id_num'] = df['user_id']

Создаем трейн из исходного ДФ, используя тестовый выше. Ключ для очистки: ***куки-вакансия-рейтинг***

In [18]:
def get_train(df,df_test):
  df['key'] = df['cookie_id_num'].astype(str)+'_'+df['vacancy_id_num'].astype(str)+'_'+df['Rating'].astype(str)
  key = set(df_test['cookie_id_num'].astype(str)+'_'+df_test['vacancy_id_num'].astype(str)+'_'+df_test['Rating'].astype(str))
  df_train = df[~df['key'].isin(key)].drop('key',axis=1)
  return df_train

Получаем и сохраняем трейн,  функция зависала в колабе, из-за нехватки памяти разбил на строки

In [11]:
df['key'] = df['cookie_id_num'].astype(str)+'_'+df['vacancy_id_num'].astype(str)+'_'+df['Rating'].astype(str)


In [19]:
key = set(df_test_top['cookie_id_num'].astype(str)+'_'+df_test_top['vacancy_id_num'].astype(str)+'_'+df_test_top['Rating'].astype(str))

In [20]:
df_train = df[~df['key'].isin(key)].drop('key',axis=1)

Сохраняем трейн и тест

In [22]:
df_test_list_top.to_csv('/content/drive/MyDrive/Hackaton 04.2023/df_test_list_top.csv',index=False)

In [26]:
df_train.to_csv('/content/drive/MyDrive/Hackaton 04.2023/df_train.csv',index=False)

In [27]:
df_train.to_parquet('/content/drive/MyDrive/Hackaton 04.2023/df_train.parquet',index=False)